# Ejercicio 10: Re-ranking

## Michael Perugachi

**Objetivo:** Implementar y evaluar un pipeline de Recuperación de Información en dos etapas, y analizar el impacto del re-ranking en la calidad del ranking.

## Parte 1. Preparación del corpus

* Cargar el corpus (documentos/pasajes).
* Cargar las consultas (queries).
* Cargar qrels (relevancia).

In [1]:
!pip install beir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.8/304.8 kB 15.6 MB/s eta 0:00:00


In [2]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader
import pandas as pd

/usr/local/lib/python3.12/dist-packages/beir/util.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
DATASET_NAME = "scifact"
DATA_DIR = "../data/beir_datasets"
url = f"https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{DATASET_NAME}.zip"
util.download_and_unzip(url, DATA_DIR)

../data/beir_datasets/scifact.zip:   0%|          | 0.00/2.69M [00:00<?, ?iB/s]

'../data/beir_datasets/scifact'

In [4]:
dataset_path = DATA_DIR + "/" + DATASET_NAME
corpus, queries, qrels = GenericDataLoader(dataset_path).load(split="test")

  0%|          | 0/5183 [00:00<?, ?it/s]

In [5]:
df_corpus = (
    pd.DataFrame.from_dict(corpus, orient="index")
      .reset_index()
      .rename(columns={"index": "doc_id"})
)

df_corpus

,doc_id,text,title
0,4983,Alterations of the architecture of cerebral wh...,Microstructural development of human newborn c...
1,5836,Myelodysplastic syndromes (MDS) are age-depend...,Induction of myelodysplasia by myeloid-derived...
2,7912,ID elements are short interspersed elements (S...,"BC1 RNA, the transcript from a master gene for..."
3,18670,DNA methylation plays an important role in bio...,The DNA Methylome of Human Peripheral Blood Mo...
4,19238,Two human Golli (for gene expressed in the oli...,The human myelin basic protein gene is include...
...,...,...,...
5178,195689316,BACKGROUND The main associations of body-mass ...,Body-mass index and cause-specific mortality i...
5179,195689757,A key aberrant biological difference between t...,Targeting metabolic remodeling in glioblastoma...
5180,196664003,A signaling pathway transmits information from...,Signaling architectures that transmit unidirec...
5181,198133135,AIMS Trabecular bone score (TBS) is a surrogat...,"Association between pre-diabetes, type 2 diabe..."


In [6]:
df_queries = (
    pd.DataFrame.from_dict(queries, orient="index", columns=["query"])
      .reset_index()
      .rename(columns={"index": "query_id"})
)

df_queries

,query_id,query
0,1,0-dimensional biomaterials show inductive prop...
1,3,"1,000 genomes project enables mapping of genet..."
2,5,1/2000 in UK have abnormal PrP positivity.
3,13,5% of perinatal mortality is due to low birth ...
4,36,A deficiency of vitamin B12 increases blood le...
...,...,...
295,1379,Women with a higher birth weight are more like...
296,1382,aPKCz causes tumour enhancement by affecting g...
297,1385,cSMAC formation enhances weak ligand signalling.
298,1389,mTORC2 regulates intracellular cysteine levels...


In [7]:
rows = []
for qid, docs in qrels.items():
    for doc_id, rel in docs.items():
        rows.append({
            "query_id": qid,
            "doc_id": doc_id,
            "relevance": rel
        })

df_qrels = pd.DataFrame(rows)
df_qrels

,query_id,doc_id,relevance
0,1,31715818,1
1,3,14717500,1
2,5,13734012,1
3,13,1606628,1
4,36,5152028,1
...,...,...,...
334,1379,17450673,1
335,1382,17755060,1
336,1385,306006,1
337,1389,23895668,1


In [8]:
# Elegimos una query cualquiera que tenga varios documentos relevantes
qid = "133"

print("Query:")
print(df_queries.loc[df_queries["query_id"] == qid, "query"].values[0])

print("\nDocumentos relevantes para esta query:")
df_qrels[(df_qrels["query_id"] == qid) & (df_qrels["relevance"] > 0)]

Query:
Assembly of invadopodia is triggered by focal generation of phosphatidylinositol-3,4-biphosphate and the activation of the nonreceptor tyrosine kinase Src.

Documentos relevantes para esta query:


,query_id,doc_id,relevance
31,133,38485364,1
32,133,6969753,1
33,133,17934082,1
34,133,16280642,1
35,133,12640810,1


## Parte 2. Retrieval inicial (baseline)

* Implementar retrieval inicial con BM25
* Obtener métricas: Recall@10 nDCG@10

In [9]:
!pip install rank_bm25

In [10]:
from rank_bm25 import BM25Okapi
from beir.retrieval.evaluation import EvaluateRetrieval
import numpy as np

# --- 1. PREPARACIÓN DE DATOS PARA BM25 ---
# BM25 necesita el texto tokenizado (dividido en palabras).
# Concatenamos 'title' y 'text' del corpus para que el modelo tenga más contexto.

# Aseguramos el orden usando listas alineadas
doc_ids = list(corpus.keys())
corpus_texts = []

for doc_id in doc_ids:
    doc = corpus[doc_id]
    # Unimos título y texto
    full_text = (doc.get("title", "") + " " + doc.get("text", "")).lower()
    corpus_texts.append(full_text.split()) # Tokenización simple por espacios

# --- 2. INDEXACIÓN (ENTRENAMIENTO DEL MODELO) ---
print("Indexando corpus con BM25...")
bm25 = BM25Okapi(corpus_texts)

# --- 3. RETRIEVAL (BÚSQUEDA) ---
print("Realizando búsqueda para las queries...")
results = {} # Diccionario para guardar resultados en formato BEIR

for qid, query_text in queries.items():
    # Tokenizamos la query igual que el corpus
    tokenized_query = query_text.lower().split()

    # Obtenemos los scores para todos los documentos
    scores = bm25.get_scores(tokenized_query)

    # BM25Okapi devuelve una lista de scores en el mismo orden que el corpus
    # Necesitamos quedarnos con los top-K (por eficiencia y formato)
    # Sin embargo, para la evaluación estándar, podemos pasar todos o un top alto.
    # Aquí mapeamos doc_id -> score

    # Optimizacion: Solo guardamos los top 100 para no llenar la memoria,
    # aunque la metrica pide @10, siempre es bueno traer un poco más.
    top_n = 100
    top_indices = np.argsort(scores)[::-1][:top_n]

    results[qid] = {doc_ids[i]: float(scores[i]) for i in top_indices}

# --- 4. EVALUACIÓN (MÉTRICAS) ---
print("Calculando métricas...")

# Usamos la clase EvaluateRetrieval de BEIR que facilita mucho esto
evaluator = EvaluateRetrieval()

# k_values define los cortes para las métricas (ej: @1, @10, @100)
ndcg, _map, recall, _p = evaluator.evaluate(qrels, results, k_values=[10])

# --- 5. RESULTADOS ---
print("\nResultados del Baseline BM25:")
print(f"Recall@10: {recall['Recall@10']:.4f}")
print(f"nDCG@10:   {ndcg['NDCG@10']:.4f}")

Indexando corpus con BM25...
Realizando búsqueda para las queries...
Calculando métricas...

Resultados del Baseline BM25:
Recall@10: 0.6862
nDCG@10:   0.5597


## Parte 3. Implementación del re-ranking _cross-encoder_

* Re-rankear los top-k candidatos para cada query.
* Identificar qué documentos cambian de posición en el top 10

In [11]:
!pip install sentence-transformers

In [12]:
from sentence_transformers import CrossEncoder
import pandas as pd

# --- 1. CARGAR EL MODELO CROSS-ENCODER ---
# Usamos 'ms-marco-MiniLM-L-6-v2', un modelo muy popular y eficiente para re-ranking
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# --- 2. LÓGICA DE RE-RANKING ---
rerank_results = {}
top_k_rerank = 50  # Solo re-rankeamos los 50 mejores que trajo BM25 para ahorrar tiempo

print(f"Iniciando Re-ranking de los top-{top_k_rerank} candidatos...")

for qid, query_hits in results.items(): # 'results' viene de tu código anterior (BM25)

    # 1. Ordenamos los resultados de BM25 y tomamos los top-k
    sorted_hits = sorted(query_hits.items(), key=lambda item: item[1], reverse=True)[:top_k_rerank]

    # 2. Preparamos los pares [Query, Documento] para el modelo
    pairs = []
    doc_ids_processed = [] # Guardamos el orden para reconstruir después

    query_text = queries[qid]

    for doc_id, score in sorted_hits:
        # Recuperamos el texto del corpus usando el doc_id
        doc_content = corpus[doc_id].get("title", "") + " " + corpus[doc_id].get("text", "")
        pairs.append([query_text, doc_content])
        doc_ids_processed.append(doc_id)

    # 3. El modelo predice la similitud (score) para todos los pares
    if len(pairs) > 0:
        cross_scores = cross_encoder.predict(pairs)

        # 4. Guardamos los nuevos resultados
        # Estructura {doc_id: new_score}
        new_scores = {doc_ids_processed[i]: float(cross_scores[i]) for i in range(len(cross_scores))}
        rerank_results[qid] = new_scores

# --- 3. COMPARACIÓN: IDENTIFICAR CAMBIOS EN EL TOP 10 ---

def get_top_n_ids(hits_dict, n=10):
    """Devuelve lista ordenada de IDs basada en scores"""
    return [k for k, v in sorted(hits_dict.items(), key=lambda item: item[1], reverse=True)[:n]]

# Usamos la misma query de ejemplo '133' para visualizar
qid_test = "133"

if qid_test in rerank_results:
    print(f"\n--- Análisis de cambios para la Query {qid_test} ---")
    print(f"Pregunta: {queries[qid_test]}")

    # Obtenemos el Top 10 original (BM25) y el nuevo (Re-ranked)
    top_bm25 = get_top_n_ids(results[qid_test], 10)
    top_rerank = get_top_n_ids(rerank_results[qid_test], 10)

    # Crear un DataFrame para ver la comparación lado a lado
    df_compare = pd.DataFrame({
        "Posición": range(1, 11),
        "Doc_ID_BM25": top_bm25,
        "Doc_ID_Reranked": top_rerank
    })

    # Añadimos columna para ver si el documento se mantuvo en la misma posición
    df_compare["Cambió?"] = df_compare["Doc_ID_BM25"] != df_compare["Doc_ID_Reranked"]

    # Añadimos si el documento en Reranked es relevante (Truth)
    relevant_docs = [doc for doc, rel in qrels[qid_test].items() if rel > 0]
    df_compare["Es_Relevante?"] = df_compare["Doc_ID_Reranked"].apply(lambda x: "SÍ" if x in relevant_docs else "No")

    print(df_compare.to_string(index=False))

    # Métrica rápida para ver si mejoró
    print("\nDocumentos Relevantes Reales:", relevant_docs)

else:
    print(f"La query {qid_test} no se encontró en los resultados.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Iniciando Re-ranking de los top-50 candidatos...

--- Análisis de cambios para la Query 133 ---
Pregunta: Assembly of invadopodia is triggered by focal generation of phosphatidylinositol-3,4-biphosphate and the activation of the nonreceptor tyrosine kinase Src.
 Posición Doc_ID_BM25 Doc_ID_Reranked  Cambió? Es_Relevante?
        1    26688294        35660758     True            No
        2     9507605        12640810     True            SÍ
        3    37964706        16280642     True            SÍ
        4     5270265         6969753     True            SÍ
        5    12785130         9507605     True            No
        6    12640810        86694016     True            No
        7    30861948        19752008     True            No
        8    86694016        17934082     True            SÍ
        9    17934082         9063688     True            No
       10     6969753        22767022     True            No

Documentos Relevantes Reales: ['38485364', '6969753', '17934082', 

## Parte 4. Implementación del re-ranking _LTR_

* Re-rankear los top-k candidatos para cada query.
* Identificar qué documentos cambian de posición en el top 10

In [14]:
!pip install xgboost scikit-learn

In [15]:
import xgboost as xgb
import pandas as pd
import numpy as np

# --- 1. PREPARACIÓN DE DATOS PARA LTR ---
# Convertimos los resultados de BM25 (diccionario) en un DataFrame plano
ltr_data = []

# Iteramos sobre los resultados de la Parte 2 (BM25)
for qid, docs in results.items():
    query_text = queries[qid]
    query_tokens = set(query_text.lower().split())

    for doc_id, bm25_score in docs.items():
        doc_obj = corpus[doc_id]
        doc_text = (doc_obj.get("title", "") + " " + doc_obj.get("text", "")).lower()
        doc_tokens = doc_text.split()

        # --- 2. INGENIERÍA DE CARACTERÍSTICAS (FEATURES) ---
        # Calculamos métricas que ayuden al modelo a distinguir relevancia

        # Feature 1: Score original de BM25
        feat_bm25 = bm25_score

        # Feature 2: Longitud del documento
        feat_doc_len = len(doc_tokens)

        # Feature 3: Conteo de palabras de la query presentes en el documento (Overlap)
        feat_overlap = sum(1 for token in query_tokens if token in doc_tokens)

        # Variable Objetivo (Label): ¿Es relevante realmente? (1 o 0)
        # Consultamos el diccionario 'qrels' (verdad fundamental)
        label = 1 if qrels[qid].get(doc_id, 0) > 0 else 0

        ltr_data.append({
            "qid": qid,
            "doc_id": doc_id,
            "bm25_score": feat_bm25,
            "doc_len": feat_doc_len,
            "term_overlap": feat_overlap,
            "label": label
        })

df_ltr = pd.DataFrame(ltr_data)

# Ordenamos por qid (Requerimiento estricto para XGBoost Ranker)
df_ltr = df_ltr.sort_values(by="qid")

# --- 3. ENTRENAMIENTO DEL MODELO (XGBRanker) ---
# Definimos las columnas que son features y cuál es el label
feature_cols = ["bm25_score", "doc_len", "term_overlap"]
X = df_ltr[feature_cols]
y = df_ltr["label"]
groups = df_ltr.groupby("qid").size().to_numpy() # Indica cuántos docs tiene cada query

print("Entrenando modelo LTR (XGBoost)...")
# Usamos 'rank:pairwise' para que el modelo aprenda a ordenar A > B
model = xgb.XGBRanker(
    objective='rank:pairwise',
    learning_rate=0.1,
    n_estimators=100,
    tree_method="hist", # Optimización para velocidad
    random_state=42
)

model.fit(X, y, group=groups, verbose=False)

# --- 4. PREDICCIÓN (RE-RANKING) ---
print("Realizando predicciones...")
# Predecimos los nuevos scores.
# NOTA: En un escenario real, deberíamos predecir sobre un set de test separado.
# Aquí re-rankeamos todo el conjunto para ver el efecto.
df_ltr["ltr_score"] = model.predict(X)

# --- 5. COMPARACIÓN: IDENTIFICAR CAMBIOS EN EL TOP 10 ---
# Usamos nuevamente la query de ejemplo "133"

qid_test = "133"
print(f"\n--- Análisis de cambios LTR para la Query {qid_test} ---")

# Filtramos los datos para esta query
df_query = df_ltr[df_ltr["qid"] == qid_test].copy()

if not df_query.empty:
    # Top 10 según BM25 original
    top_bm25_ids = df_query.sort_values(by="bm25_score", ascending=False).head(10)["doc_id"].tolist()

    # Top 10 según LTR (Nuestra nueva predicción)
    top_ltr_ids = df_query.sort_values(by="ltr_score", ascending=False).head(10)["doc_id"].tolist()

    # Creamos tabla comparativa
    df_compare_ltr = pd.DataFrame({
        "Posición": range(1, 11),
        "Doc_ID_BM25": top_bm25_ids,
        "Doc_ID_LTR": top_ltr_ids
    })

    # Verificamos cambios y relevancia
    df_compare_ltr["Cambió?"] = df_compare_ltr["Doc_ID_BM25"] != df_compare_ltr["Doc_ID_LTR"]

    # Check de relevancia real (Ground Truth)
    relevant_docs = [doc for doc, rel in qrels[qid_test].items() if rel > 0]
    df_compare_ltr["Es_Relevante?"] = df_compare_ltr["Doc_ID_LTR"].apply(lambda x: "SÍ" if x in relevant_docs else "No")

    print(df_compare_ltr.to_string(index=False))

    # Ver importancia de características (Qué usó el modelo para decidir)
    print("\nImportancia de las características:")
    print(pd.Series(model.feature_importances_, index=feature_cols).sort_values(ascending=False))

else:
    print(f"No se encontraron datos LTR para la query {qid_test}")

Entrenando modelo LTR (XGBoost)...
Realizando predicciones...

--- Análisis de cambios LTR para la Query 133 ---
 Posición Doc_ID_BM25 Doc_ID_LTR  Cambió? Es_Relevante?
        1    26688294   26688294    False            No
        2     9507605   17934082     True            SÍ
        3    37964706   12640810     True            SÍ
        4     5270265    5821617     True            No
        5    12785130   30224907     True            No
        6    12640810    7451607     True            No
        7    30861948    5270265     True            No
        8    86694016   86694016    False            No
        9    17934082    9063688     True            No
       10     6969753   16280642     True            SÍ

Importancia de las características:
bm25_score      0.659339
term_overlap    0.197306
doc_len         0.143354
dtype: float32


## Parte 5. Evaluación post re-ranking

Calcular métricas:
* nDCG@10
* MAP
* Recall@10

In [17]:
from beir.retrieval.evaluation import EvaluateRetrieval

# Inicializamos el evaluador
evaluator = EvaluateRetrieval()

# ---------------------------------------------------------
# 1. EVALUACIÓN DEL BASELINE (BM25) - Parte 2
# ---------------------------------------------------------
# Usamos 'results' que contiene los scores originales de BM25
print("--- Evaluando BM25 (Baseline) ---")
ndcg_bm25, map_bm25, recall_bm25, _ = evaluator.evaluate(qrels, results, k_values=[10])

# ---------------------------------------------------------
# 2. EVALUACIÓN DEL CROSS-ENCODER - Parte 3
# ---------------------------------------------------------
print("\n--- Evaluando Cross-Encoder ---")
ndcg_ce, map_ce, recall_ce, _ = evaluator.evaluate(qrels, rerank_results, k_values=[10])

# ---------------------------------------------------------
# 3. EVALUACIÓN DEL LTR (XGBoost) - Parte 4
# ---------------------------------------------------------
print("\n--- Evaluando LTR (XGBoost) ---")
ltr_results = {}
# Reconstruimos el diccionario desde el DataFrame
for qid, group in df_ltr.groupby("qid"):
    ltr_results[str(qid)] = dict(zip(group["doc_id"], group["ltr_score"]))

ndcg_ltr, map_ltr, recall_ltr, _ = evaluator.evaluate(qrels, ltr_results, k_values=[10])

# ---------------------------------------------------------
# 4. TABLA COMPARATIVA FINAL
# ---------------------------------------------------------
print("\n" + "="*65)
print(f"{'MÉTODO':<20} | {'nDCG@10':<10} | {'Recall@10':<10} | {'MAP@10':<10}")
print("-" * 65)

# Fila 1: BM25
print(f"{'BM25 (Baseline)':<20} | {ndcg_bm25['NDCG@10']:.4f}     | {recall_bm25['Recall@10']:.4f}     | {map_bm25['MAP@10']:.4f}")

# Fila 2: LTR
print(f"{'LTR (XGBoost)':<20} | {ndcg_ltr['NDCG@10']:.4f}     | {recall_ltr['Recall@10']:.4f}     | {map_ltr['MAP@10']:.4f}")

# Fila 3: Cross-Encoder
print(f"{'Cross-Encoder':<20} | {ndcg_ce['NDCG@10']:.4f}     | {recall_ce['Recall@10']:.4f}     | {map_ce['MAP@10']:.4f}")

print("="*65)

--- Evaluando BM25 (Baseline) ---

--- Evaluando Cross-Encoder ---

--- Evaluando LTR (XGBoost) ---

MÉTODO               | nDCG@10    | Recall@10  | MAP@10    
-----------------------------------------------------------------
BM25 (Baseline)      | 0.5597     | 0.6862     | 0.5147
LTR (XGBoost)        | 0.7676     | 0.7923     | 0.7543
Cross-Encoder        | 0.6444     | 0.7402     | 0.6077
